In [1]:
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from sklearn.cluster import DBSCAN
from imutils.video import FPS
from imutils import paths
import face_recognition
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os

## Setup Video Agent

In [2]:
class VideoAgent:
    def __init__(self):
        print("[INFO] starting clustering agent...")
        self.clt = DBSCAN(min_samples=20, metric='euclidean', eps= 0.44, n_jobs=-1)
            
        print("[INFO] starting video stream...")
        self.video_capture = cv2.VideoCapture("./sample_videos/videoplayback.mp4")

        self.encodings = []
        self.frames = []
        self.frame_count = 0
        self.labelSet = set()

In [58]:
va = VideoAgent()

[INFO] clustering agent...
[INFO] starting video stream...


## Main

In [55]:
def main(va):
    clt = va.clt
    video_capture = va.video_capture
    encodings = va.encodings
    frames = va.frames
    frame_count = va.frame_count
    labelSet = va.labelSet
    
    while video_capture.isOpened():
    # for asdfas in range(10): # JUST FOR TESTS

        # Grab a single frame of video
        ret, frame = video_capture.read()
        # frame = imutils.resize(frame, width=600)

        frame_count += 1
        if(frame_count%24 == 0):
            frame_count = 0

        else:
            # frame = imutils.resize(frame, width=600)  
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            boxes = face_recognition.face_locations(rgb,model="cnn")

            encodingsSize = len(encodings)

            # compute the facial embedding for the face
            encoding = face_recognition.face_encodings(rgb, boxes)
            if encoding:
                encodings += encoding
                clt.fit(encodings)
                for i in clt.fit_predict(encodings):
                    if i!=-1 :
                        labelSet.add(i)


            for i, box in enumerate(boxes):
                # coordinates
                (top, right, bottom, left) = box
                y = top - 10 if top - 10 > 10 else top + 10

                # perform classification to recognize the face
                name = "ID"
                text = "{}: {}".format(name, clt.labels_[encodingsSize+i])

                # plot at image
                cv2.rectangle(frame, (right,top), (left, bottom), (0, 0, 255), 2)
                cv2.putText(frame, text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

            cv2.putText(frame, 'Counting: {}'.format(len(labelSet)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 200, 200), 2)

            # show the output frame
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

    # do a bit of cleanup
    cv2.destroyAllWindows()

In [62]:
main(va)